# Noise to cascade evaluation

this notebook is thought to provide an overview that compares the noise records with the event records and combined records to check which records to include for training

In [ ]:
from ananke.models.collection import Collection

In [ ]:
cascades_collection = Collection('../../data/cascades_30000/data.h5', read_only=True)
noise_collection = Collection('../../data/electrical_noise_100000/data.h5', read_only=True)
bioluminescense_collection = Collection('../../olympus/examples/data/bioluminescence_100/data.h5', read_only=True)
combined_noise_collection = Collection('../../data/merge/combined_noise_overlap_10000.h5', read_only=True)

In [ ]:
cascades_record_statistics = cascades_collection.get_record_statistics()
cascade_hit_counts = cascades_record_statistics.df['hit_count']

In [ ]:
noise_record_statistics = noise_collection.get_record_statistics()
noise_hit_counts = noise_record_statistics.df['hit_count']

In [ ]:
bioluminescence_record_statistics = bioluminescense_collection.get_record_statistics()
bioluminescence_hit_counts = bioluminescence_record_statistics.df['hit_count']

In [ ]:
combined_noise_record_statistics = combined_noise_collection.get_record_statistics()
combined_noise_hit_counts = combined_noise_record_statistics.df['hit_count']

In [ ]:
cascade_hit_counts = cascades_record_statistics.df['hit_count']

In [ ]:
print(len(cascade_hit_counts))
cascade_hit_counts.plot.hist(bins=100, logy=True, xlim=(0,10000000))

In [ ]:
noise_hit_counts.plot.hist(bins=20)

With these statistics, we can tell that after a given amount of hits, it is not really probable that it is a noise event. Let's go for 5 sigma

In [ ]:
bioluminescence_hit_counts.plot.hist(bins=100, xlim=(0,1000))

In [ ]:
combined_noise_hit_counts.plot.hist(bins=100)

In [ ]:

one_sigma = 0.682689492137064
three_sigma = 0.997300203936740
five_sigma = 0.999999426696856


combined_noise_quantile = combined_noise_hit_counts.quantile(one_sigma)

In [ ]:
noise_mean = noise_hit_counts.mean()
noise_std = noise_hit_counts.std()

In [ ]:
five_sigma = noise_mean + 5 * noise_std
five_sigma

Based on 100.000 electronical noise records, everything with more than 87 hits is most likely to be a event. Hence, we can limit our efforts on events below this threshold.

In [ ]:
feasible_cascades_df = cascades_record_statistics.df
feasible_cascades_df = feasible_cascades_df[feasible_cascades_df['hit_count'] < combined_noise_three_sigma]

In [ ]:
feasible_cascades_df.head()

In [ ]:
len(feasible_cascades_df.index)

In [ ]:
feasible_cascades_df.reset_index().to_feather('../../data/combined_noise_feasable_cascades.feather')

In [43]:
len(feasible_cascades_df.index)

2290

In [45]:
feasible_cascades_df.reset_index().to_feather('../../data/combined_noise_feasable_cascades.feather')